In [10]:
import pandas as pd
import numpy as np
import pymysql

In [36]:
def convertir_a_meses(periodo):
    # Dividimos la cadena en partes
    partes = periodo.split()
    
    # Inicializamos los años y meses a 0
    años = 0
    meses = 0
    
    # Iteramos sobre las partes y extraemos los valores de años y meses
    for i in range(len(partes)):
        if partes[i] == "AÑOS":
            años = int(partes[i - 1])
        elif partes[i] == "MESES":
            meses = int(partes[i - 1])
    
    # Convertimos todo a meses
    total_meses = años * 12 + meses
    return total_meses

In [69]:
cnx = pymysql.connect(host='159.89.90.197', port = 3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')
cursor = cnx.cursor()

In [29]:
replace_dict_oficios = {'jarcieria': 'servicios',
 'tlapaleria/ferreteria/materiales de construccion': 'servicios',
 'lavanderia/tintoreria/planchaduria': 'servicios',
 'combustibles/gas/gasolina': 'servicios',
 'animales/veterinaria': 'servicios',
 'desperdicios/residuos/reciclaje': 'bienes de consumo',
 'calzado reparacion y venta': 'bienes de consumo',
 'venta por catalogo': 'ventas',
 'venta de ropa': 'ventas',
 'distribuidor con experiencia': 'ventas',
 'venta de alimentos': 'venta de alimentos',
 'belleza/salon/estetica/barberia': 'estetica',
 'joyeria/bisuteria/fundicion': 'estetica',
 'eventos/salones/sillas/mesas': 'eventos',
 'flores/arreglos': 'eventos',
 'fotografia/video': 'eventos',
 'imprenta/serigrafia/rotulos': 'papeleria',
 'cafeinternet': 'papeleria',
 'papeleria': 'papeleria',
 'agua/potable/residual': 'bienes de consumo',
 'carniceria/polleria/rastro': 'bienes de consumo',
 'tortilleria/molino': 'bienes de consumo',
 'frutas/verduras/semillas/campo/agropecuarias': 'bienes de consumo',
 'abarrotes/miscelaneas/perecederos': 'bienes de consumo',
 'panaderia': 'bienes de consumo',
 'amedicina/salud/farmacia': 'salud',
 'deportes/gimnasio': 'prestamo de servicios',
 'educacion/escuelas/centro de capacitacion': 'prestamo de servicios',
 'venta de autos/motocicletas/lotes/autopartes': 'venta de electronicos/automoviles',
 'venta de celulares/tecnologia': 'venta de electronicos/automoviles',
 'venta de muebles/electronicos/electrodomesticos': 'venta de electronicos/automoviles',
 'electricista': 'oficios',
 'plomeria': 'oficios',
 'herreria': 'oficios',
 'cerrajeria': 'oficios',
 'arte/artesanias': 'oficios',
 'fabrica/taller/elaboracion/reparacion': 'oficios',
 'madera/carpinteria/ebanista': 'oficios',
 'mecanica automotriz/industrial': 'oficios'}

replace_dict_sucursal = {
    "amecameca": "Grupo 1",
    "milpa alta": "Grupo 1",
    "tlahuac": "Grupo 1",
    "coatzacoalcos": "Grupo 3",
    "fortin": "Grupo 3",
    "orizaba": "Grupo 3",
    "texcoco": "Grupo 2",
    "texcoco 2": "Grupo 2",
    "villahermosa": "Grupo 2",
    "chalco": "Grupo 2",
    "cardenas": "Grupo 2",
    "ixtapaluca": "Grupo 2",
    "iztapalapa": "Grupo 4",
    "nezahualcoyotl": "Grupo 4",
    "nezahualcoyotl 2": "Grupo 4",
    "chimalhuacan 2": "Grupo 4",
    "tonala": "Grupo 5",
    "tonala 2": "Grupo 5"
}


In [2]:
data_pc = pd.read_excel("./data/240221 ANALISIS DE MODELOS.xlsx", sheet_name='Hoja1')

In [9]:
data_pc.columns = [str(k).lower().replace(" ", "_") for k in data_pc.columns]

data_pc = data_pc[['id_distribuidor', 'p_cosecha']]

data_pc.head(1)

,id_distribuidor,p_cosecha
0,145442,0.0


In [11]:
ids = ", ".join(data_pc.id_distribuidor.unique().astype(str))

In [60]:
sql = f"""
select *  from 
	(select id,  escolaridad, lower(sucursal) as sucursal, lower(tipo_negocio) as tipo_negocio, tiempo_op_negocio, compras, 
		ingreso_mensual_negocio, ventas from cc_prospectos_demograficos
UNION        
	select id,  escolaridad, lower(sucursal) as sucursal, lower(tipo_negocio) as tipo_negocio, tiempo_op_negocio, compras, 
		ingreso_mensual as 'ingreso_mensual_negocio', ventas from demograficos_distribuidores
) as un
where un.id in ({ids})
"""
cursor.execute(sql)
data_demo = cursor.fetchall()
data_demo = pd.DataFrame(data_demo, columns=[i[0] for i in cursor.description])
data_demo.rename(columns={"id" : 'id_distribuidor'}, inplace=True)
data_demo.id_distribuidor = data_demo.id_distribuidor.astype(int)



In [61]:
data_demo.tipo_negocio = data_demo.tipo_negocio.apply(lambda x: replace_dict_oficios[x].replace(" ","_").upper())
data_demo.tiempo_op_negocio = data_demo.tiempo_op_negocio.apply(convertir_a_meses)

In [62]:
data_demo.sucursal = data_demo.sucursal.apply(lambda x: replace_dict_sucursal[x].replace(" ","_").upper())

In [63]:
data_demo.ingreso_mensual_negocio = data_demo.ingreso_mensual_negocio.apply(lambda x: float(x.replace("$", "").replace(",", "")))

In [71]:
sql = f"""
select 
		nombreOtorgante as 'NOMBREOTORGANTE', 
		pc.folioConsulta as 'FOLIO', 
        importeCredito as 'IMPORTECREDITO', 
        fechaConsulta,
        idCliente as 'id_distribuidor',
        valor as 'SCORE_FICO' from cc_prospectos_consultas as pc
left join cc_prospectos_folios as pf on pc.folioConsulta = pf.folioConsulta 
left join cc_prospectos_scores as ps on pc.folioConsulta = ps.folioConsulta 
where idCliente in ({ids}) and fechaConsulta >= DATE_SUB(NOW(), INTERVAL 12 MONTH)
"""

In [72]:
cursor.execute(sql)
data_cc = cursor.fetchall()
data_cc = pd.DataFrame(data_cc, columns=[i[0] for i in cursor.description])
data_cc.rename(columns={"id" : 'id_distribuidor'}, inplace = True)
data_cc.id_distribuidor = data_cc.id_distribuidor.astype(int)

In [85]:
data_enviar = data_demo.merge(data_cc, on='id_distribuidor')

In [102]:
data_enviar.ESCOLARIDAD = data_enviar.ESCOLARIDAD.str.upper()

In [103]:
data_enviar.columns = [k.upper() for k in data_enviar.columns]

data_enviar.rename(columns={"ID_DISTRIBUIDOR":"APPLICANT_ID"}, inplace=True)

In [104]:
data_enviar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12774 entries, 0 to 12773
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   APPLICANT_ID             12774 non-null  int32  
 1   ESCOLARIDAD              12774 non-null  object 
 2   SUCURSAL                 12774 non-null  object 
 3   TIPO_NEGOCIO             12774 non-null  object 
 4   TIEMPO_OP_NEGOCIO        12774 non-null  int64  
 5   COMPRAS                  12774 non-null  float64
 6   INGRESO_MENSUAL_NEGOCIO  12774 non-null  float64
 7   VENTAS                   12774 non-null  int64  
 8   NOMBREOTORGANTE          12774 non-null  object 
 9   FOLIO                    12774 non-null  object 
 10  IMPORTECREDITO           12774 non-null  int64  
 11  FECHACONSULTA            12774 non-null  object 
 12  SCORE_FICO               12774 non-null  int64  
dtypes: float64(2), int32(1), int64(4), object(6)
memory usage: 1.3+ MB


In [105]:
data_pc_enviar = data_pc.loc[data_pc.id_distribuidor.isin(data_enviar.APPLICANT_ID)]

In [106]:
data_enviar.to_csv("./data/enviar/data_backtesting.csv", index=False)
data_pc_enviar.to_csv("./data/enviar/data_pc_backtesting.csv", index=False)

In [108]:
data_pc_enviar.id_distribuidor

2       157169
3       157171
4       157178
5       157185
6       157188
         ...  
1412    167070
1413    167202
1414    166739
1415    166784
1416    166882
Name: id_distribuidor, Length: 1184, dtype: int64

In [97]:
cnx.close()